## data preparation

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices())

2023-11-05 20:58:51.742977: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 20:58:51.765973: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-05 20:58:51.765992: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-05 20:58:51.766013: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-05 20:58:51.770474: I tensorflow/core/platform/cpu_feature_g

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-11-05 20:58:52.778907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 20:58:52.781633: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 20:58:52.781705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
import sys
ABS_ROOT_DIR_PATH = '/host/dissertation/'
if ABS_ROOT_DIR_PATH not in sys.path:
    sys.path.append(ABS_ROOT_DIR_PATH)

In [3]:
POINTS_NUM = 10
CLASSES_NUM = 2

In [4]:
import numpy as np
points = np.load(f'/host/dissertation/proccessed_data/points_{POINTS_NUM}.npy')
q = np.load('/host/dissertation/proccessed_data/q.npy')

In [5]:
q[q == -1] = 0
q_labels = tf.one_hot(q, CLASSES_NUM)

2023-11-05 20:58:52.950268: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 20:58:52.950749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 20:58:52.951101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## manual testing

In [6]:
import cvnn.layers as complex_layers

model = tf.keras.models.Sequential()
model.add(complex_layers.ComplexInput(input_shape=(POINTS_NUM,)))
model.add(complex_layers.ComplexDense(units=64, activation='pol_sigmoid'))
model.add(complex_layers.ComplexDense(units=64, activation='pol_sigmoid'))
model.add(complex_layers.ComplexDense(units=CLASSES_NUM, activation='softmax_real_with_abs'))
model.compile(optimizer=tf.keras.optimizers.SGD(clipnorm=1.), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [7]:
from common.model import FreeMemory
history = model.fit(points, q_labels, epochs=350, validation_split=0.2, verbose=0, callbacks=[FreeMemory(10)])

In [8]:
from common.plot import plot_metric_history
plot_metric_history(history, metric='accuracy', moving_average_window=(20,))

## space search

In [9]:
from common.search import ConfigsTester

configs_tester = ConfigsTester(spacepoints_number_space = [10,6,3,],
                               hiddenlayers_number_space = [2,3,1],
                               hiddenlayers_neurons_number_space = [64,32,128,],
                               hiddenlayers_activation_func_space = ['pol_sigmoid','pol_tanh','cart_relu',],
                               outputlayer_activation_func_space = ['sigmoid_real', 'softmax_real_with_abs', 'softmax_real_with_polar', 'softmax_real_with_avg', 'softmax_real_with_mult', 'softmax_of_softmax_real_with_avg', 'softmax_of_softmax_real_with_mult' ],
                               metric = 'accuracy')

In [10]:
import cvnn.layers as complex_layers
from common.model import FreeMemory

configs_tester.test_configs(y_data=q_labels,
                            optimizer_factory=lambda: tf.keras.optimizers.SGD(clipnorm=1.), 
                            loss='binary_crossentropy', 
                            epochs=250, 
                            callbacks=[FreeMemory()])

START (10, 2, 64, 'pol_sigmoid', 'sigmoid_real')
      model compiled
      training...


2023-11-05 20:58:55.929094: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86380391f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-05 20:58:55.929115: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2023-11-05 20:58:56.016940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-11-05 20:58:56.049092: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


      model trained
      hp_file appended
      history saved
      weights saved
DONE  (10, 2, 64, 'pol_sigmoid', 'sigmoid_real')
START (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_abs')
      model compiled
      training...
      model trained
      hp_file appended
      history saved
      weights saved
DONE  (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_abs')
START (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_polar')
      model compiled
      training...
      model trained
      hp_file appended
      history saved
      weights saved
DONE  (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_polar')
START (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_avg')
      model compiled
      training...
      model trained
      hp_file appended
      history saved
      weights saved
DONE  (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_avg')
START (10, 2, 64, 'pol_sigmoid', 'softmax_real_with_mult')
      model compiled
      training...
      model trained
      hp_file appended
      his

KeyboardInterrupt: 